In [1]:
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from gezi.common import *
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()
gezi.set_pandas()

In [2]:
V = 7
root = f'../working/offline/{V}/0'
# pairwise two tower, recall model
# p2t_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_ext_emlm_mlm.ep1.eval'
# p2t_model = 'all-mpnet-base-v2.flag-pairwise14-2'
p2t_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_mlm3'
p2t_model2 = 'pmminilm.flag-pairwise14-2-pre_emlm_mlm-mmnilm'
p2t_model2 = 'pmminilm.flag-pairwise14-2'
# pairwise concat, rank model
pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-extpred-ft.neg_rand_prob-0.neg_strategy-rand-sample.eval'
# context model
context_model = 'deberta-v3-small.flag-context4-2-d'
context_model2 = 'lsg-mminilm.flag-context4-3-d-s-mminilm'

In [22]:
df = pd.read_feather('../working/train.fea')

In [23]:
df.columns

Index(['cell_id', 'cell_type', 'source', 'id', 'n_words', 'cid', 'ancestor_id',
       'parent_id', 'n_cell', 'n_code_cell', 'n_markdown_cell',
       'markdown_frac', 'rank', 'code_rank', 'markdown_rank', 'rel_rank',
       'global_rank', 'local_rank', 'pct_rank', 'fold', 'worker'],
      dtype='object')

In [24]:
d = df[['id', 'cell_id', 'rank', 'cell_type', 'rel_rank']]
d

,id,cell_id,rank,cell_type,rel_rank
0,00001756c60be8,1862f0a6,0,code,0.0323
1,00001756c60be8,448eb224,1,markdown,0.0484
2,00001756c60be8,2a9e43d6,2,code,0.0645
3,00001756c60be8,7e2f170a,3,markdown,0.0806
4,00001756c60be8,038b763d,4,code,0.0968
...,...,...,...,...,...
6370641,fffe1d764579d5,5369934b,67,markdown,0.9597
6370642,fffe1d764579d5,ac9db030,68,code,0.9677
6370643,fffe1d764579d5,a8ffc8b4,69,markdown,0.9758
6370644,fffe1d764579d5,70455170,70,code,0.9839


In [25]:
xp = gezi.load(f'{root}/{p2t_model}/valid.pkl')
dxp = pd.DataFrame(xp)

In [7]:
# dxp

In [8]:
calc_metric(xp)

0.8906600693426994

In [9]:
calc_metric(xp, 'cls_pred')

0.8940493638363678

In [10]:
scores = calc_metrics(xp, 'pred')
# scores

calc_metrics:   0%|          | 0/27590 [00:00<?, ?it/s]

In [30]:
dxp = to_df(xp, groupby=False, key='pred')

In [31]:
# dxp

In [32]:
dxp = dxp[['id', 'cell_id', 'rank', 'cell_type', 'rel_rank', 'pred']]

In [33]:
scores = scores.sort_values('score')

In [34]:
dxp.loc[dxp["cell_type"] == 'code',
        'pred'] = dxp.loc[dxp["cell_type"] == 'code', 'rel_rank']

In [35]:
def vis(idx):
  # print(scores.score.values[idx])
  id = scores.id.values[idx]
  id = '4562d150cadf6a'
  id = '1a7d96060f8e36'
  id = '674983f1fb6a94'
  id = '2d93ad6380413d'
  id = 'a6073f4fdf1845'
  d2 = dxp[dxp.id==id]
  ic(scores[scores.id==id])
  gezi.plot.display_side_by_side((d[d.id==id], 'gt'), (d2.sort_values('pred'), 'pred'))
  d_ = df[['id', 'cell_id', 'rank', 'cell_type', 'rel_rank', 'source']]
  d_ = d_[d_.id==id]
  display(d_)

In [36]:
vis(1000)

[08/08/22 21:32:10] 1115278365.py:10 in vis()
                    scores[scores.id==id]:                    id  score
                                           19754  a6073f4fdf1845 0.8794


,id,cell_id,rank,cell_type,rel_rank
4144473,a6073f4fdf1845,58c2cc54,0,markdown,0.0172
4144474,a6073f4fdf1845,3c8ef895,1,code,0.0345
4144475,a6073f4fdf1845,1e525ebb,2,code,0.0690
4144476,a6073f4fdf1845,5681077e,3,markdown,0.0862
4144477,a6073f4fdf1845,290ae46d,4,code,0.1034
4144478,a6073f4fdf1845,ab7ab2ad,5,markdown,0.1207
4144479,a6073f4fdf1845,e708def2,6,code,0.1379
4144480,a6073f4fdf1845,635c05d1,7,code,0.1724
4144481,a6073f4fdf1845,11b29874,8,markdown,0.1897
4144482,a6073f4fdf1845,65c3c0f0,9,code,0.2069


,id,cell_id,rank,cell_type,rel_rank,source
4144473,a6073f4fdf1845,58c2cc54,0,markdown,0.0172,# Playing Surface Analysis\n![](https://i.imgur.com/ZsERLAT.png)\n## Lateral movement and increased injury risk.\n\nThis report contains my analysis for the 2019 NFL 1st and Future competition. The competition tasked data scientists to investigate the relationship between the playing surface and the injury and performance of National Football League (NFL) athletes and to examine factors that may contribute to lower extremity injuries.\n\nI propose a metric for measuring the angle between ath...
4144474,a6073f4fdf1845,3c8ef895,1,code,0.0345,"import pandas as pd\nimport numpy as np\nimport seaborn as sns\nimport matplotlib.pylab as plt\nimport os\nimport psutil\n\nimport lightgbm as lgb\nfrom sklearn.model_selection import train_test_split\nimport random\nimport sklearn\nfrom itertools import cycle, islice\n\nimport warnings\nwarnings.filterwarnings(""ignore"")\n\nfrom tqdm.notebook import tqdm\npd.set_option('max_columns', 500)\nplt.style.use('fivethirtyeight')"
4144475,a6073f4fdf1845,1e525ebb,2,code,0.0690,"# Read in data\ntracks = pd.read_csv('../input/nfl-playing-surface-analytics/PlayerTrackData.csv',\n dtype={'time':'float64',\n 'x':'float16',\n 'y':'float16',\n 'dir': 'float16',\n 'dis': 'float16',\n 'o':'float16',\n 's':'float16'})\n\nplays = pd.read_csv('../input/nfl-playing-surface..."
4144476,a6073f4fdf1845,5681077e,3,markdown,0.0862,"# Background\nThere has long been professional and academic interest in the differences regarding athletic performance measures and injury occurrence on various surface types (Meyers and Barnhill, 2004; Powell and Schootman, 1992). Recent studies have specifically examined synthetic turf (Mack et al., 2018; Loughran et al., 2019). These observational studies only show a correlation in injury rate-- they don’t make assertions about the specific mechanisms associated with lower body injury. My..."
4144477,a6073f4fdf1845,290ae46d,4,code,0.1034,"plt.style.use('fivethirtyeight')\n# Find Injury Rate by Surface\ninjury_playkeys = injury['PlayKey'].unique()\nplays['counter'] = 1 # Column used when grouping to count\nplays['isInjuryPlay'] = False\nplays.loc[plays['PlayKey'].isin(injury_playkeys), 'isInjuryPlay'] = True\n\n# Plot Results\nfig, ax = plt.subplots(1, 1, figsize=(8, 6))\n(plays.groupby('FieldType')[['isInjuryPlay']].mean() * 100000).plot(kind='bar', ax=ax)\nax.get_legend().remove()\nax.set_xlabel('')\nax.set_title('Lower Body..."
4144478,a6073f4fdf1845,ab7ab2ad,5,markdown,0.1207,"# Focusing on the players and plays at high risk\n\nIt’s intuitive, and quickly reveals itself in the data that certain players are at higher risk than others when it comes to NC lower body injuries. Among the 100+ injuries from the 2 seasons of game play, 70% (74/105) were sustained by just three positions (Defensive Backs, Linebackers and Wide Receivers). When thinking about why these positions might be at higher risk, I envisioned the movements commonly made by these players. I was stru..."
4144479,a6073f4fdf1845,e708def2,6,code,0.1379,"#### USED FOR SLIDES\n\n# ax = injury_detailed.groupby('RosterPosition_notplay')['PlayerKey'].count().sort_values() \\n# .plot(kind='barh', title='Non-Contact Injuries', figsize=(8, 5))\n# count = 0\n# for x in ax.patches:\n# if count > 3:\n# x.set_color('orange')\n# count += 1\n# # plt.annotate('Three positions account for 2/3 of injuries', xy=(15, 1.5), fontsize=15, color='brown')\n# # plt.arrow(15, 2, -0.3, 1, color='brown', head_width=0.2, head_length=0.2, lw=3)\n# ax..."
4144480,a6073f4fdf1845,635c05d1,7,code,0.1724,"fig, axes = plt.subplots(1, 2, figsize=(15, 5))\nax = axes[0]\ninjury_detailed.groupby('RosterPosition_notplay')['PlayerKey'].count().sort_values() \\n .plot(kind='barh', title='Non-Contact Injuries', figsize=(10, 5), ax=ax)\ncount = 0\nfor x in ax.patches:\n if count > 3:\n x.set_color('orange')\n count += 1\n# plt.annotate('Three positions account for 2/3 of inju

In [37]:
# xp = gezi.load(f'{root}/{p2t_model}/valid.pkl')
# d = pd.DataFrame(xp)
# d[d.id=='1a7d96060f8e36']